# Small

In [1]:
import simuOpt
simuOpt.setOptions(alleleType='short', numThreads=4, quiet=True)
import simuPOP as sim
import pandas as pd
from saegus import breed, operators, simulate, analyze, parse, parameters
import shelve
import numpy as np
import random
import h5py
import collections as col
np.set_printoptions(suppress=True, precision=3)
pd.options.display.float_format = '{:.4f}'.format

In [2]:
small_data = h5py.File('small_data.hdf5')

In [3]:
small = analyze.Study('small', number_of_replicates=10, data_file=small_data)

In [4]:
run_id = 'small'
generations_of_random_mating = 10
number_of_qtl = 20
number_of_replicates = 10
founders = [[2, 26], [3, 25], [4, 24], [5, 23]]
os_per_pair = 500
recombination_rates = [0.01]*1478

In [5]:
prefounders = sim.loadPopulation('bia_prefounders.pop')

In [6]:
prefounders.infoFields()

('ind_id',
 'father_id',
 'mother_id',
 'fitness',
 'p',
 'g',
 'generation',
 'replicate')

In [7]:
sim.tagID(prefounders, reset=True)

In [8]:
prefounders.popSize()

26

In [9]:
multi_prefounders = sim.Simulator(prefounders, 10, stealPops=False)

In [10]:
magic = breed.MAGIC(multi_prefounders, founders, recombination_rates)

In [11]:
magic.generate_f_one(founders, os_per_pair)

In [12]:
mrc = breed.MultiRandomCross(multi_prefounders, 4, 500)

In [13]:
mother_choices, father_choices = mrc.determine_random_cross()

In [14]:
multi_snd_ord_chooser = breed.MultiSecondOrderPairIDChooser(
    mother_choices, father_choices)

In [15]:
multi_prefounders.evolve(
    matingScheme=sim.HomoMating(
        sim.PyParentsChooser(multi_snd_ord_chooser.snd_ord_id_pairs),
        sim.OffspringGenerator(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=0.01)
        ],
            numOffspring=1),
        subPopSize=[2000],
    ),
    gen=1,
)

(1, 1, 1, 1, 1, 1, 1, 1, 1, 1)

In [16]:
final_mrc = breed.MultiRandomCross(multi_prefounders, 2, 1000)

In [17]:
final_mothers, final_fathers = final_mrc.determine_random_cross()

In [18]:
final_multi_snd_ord_chooser = breed.MultiSecondOrderPairIDChooser(
    final_mothers, final_fathers)

In [19]:
multi_prefounders.evolve(
    matingScheme=sim.HomoMating(
        sim.PyParentsChooser(final_multi_snd_ord_chooser.snd_ord_id_pairs),
        sim.OffspringGenerator(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=0.01)
        ],
            numOffspring=1),
        subPopSize=[2000],
    ),
    gen=1,
)

(1, 1, 1, 1, 1, 1, 1, 1, 1, 1)

# Random Mating Phase

In [20]:
multi_prefounders.evolve(
    matingScheme=sim.RandomMating(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=0.01)
        ],
        subPopSize=[2000]),
    gen=10,
)

(10, 10, 10, 10, 10, 10, 10, 10, 10, 10)

In [21]:
sample_size = 500

In [22]:
sample_library = small.collect_samples(multi_prefounders, [sample_size])

In [23]:
for rep_id, sample_list in sample_library.items():
    sim.stat(sample_list[0], numOfSegSites=sim.ALL_AVAIL, vars=['numOfSegSites', 'segSites'])
    sim.stat(sample_list[0], alleleFreq=sim.ALL_AVAIL)

In [24]:
sample = sample_library[0][0]

In [25]:
astates = small.gather_allele_data(sample)

In [26]:
alleles = np.array([astates[:, 1], astates[:, 2]]).T

In [27]:
segregating_loci = np.array(sample.dvars().segSites)

In [28]:
trait = parameters.Trait()

In [29]:
qtl = sorted(list(random.sample(list(segregating_loci), 20)))

In [30]:
allele_effects = trait.construct_allele_effects_table(alleles, qtl, random.expovariate, 1)

In [31]:
ae_array = trait.construct_ae_array(allele_effects, qtl)

# Storing Data

In [32]:
small_data['allele/states'] = astates
small_data['segregating_loci'] = segregating_loci
small_data['qtl'] = np.array(qtl)
small_data['allele/effects'] = allele_effects
small_data['recombination_rates'] = np.array(recombination_rates)
small_data['allele/effects_array'] = ae_array

In [33]:
for rep, sample_list in sample_library.items():
    small_data['allele/frequency/replicate/' + str(rep)] = small.gather_allele_frequencies(sample_list[0], astates)
    operators.calculate_g(sample_list[0], ae_array)
    operators.calculate_error_variance(sample_list[0], 0.7)
    operators.calculate_p(sample_list[0])
    small_data['trait/g/replicate/' + str(rep)] = np.array([sample_list[0].indInfo('ind_id'), 
                                                            sample_list[0].indInfo('g')]).T
    small_data['trait/p/replicate/' + str(rep)] = np.array([sample_list[0].indInfo('ind_id'),
                                                          sample_list[0].indInfo('p')]).T
    

In [34]:
small_data['trait'].attrs['heritability'] = np.array([0.7])

# Correcting the Config File

In [35]:
minor_alleles = np.array(small_data['allele/states'])[:, 3]

In [ ]:
name = 'small_0'

In [36]:
from os import path

In [37]:
indir = '/home/vakanas/tassel-5-standalone/input'
outdir = '/home/vakanas/tassel-5-standalone/output'

In [38]:
for rep, sample_list in sample_library.items():
    name = small.run_id + '_' + str(rep)
    minor_allele_fs = np.array(small_data['allele/frequency/replicate/' + str(rep)])[segregating_loci, 3]
    gwas = analyze.GWAS(sample_list[0], segregating_loci, minor_alleles, 'small')
    cm = gwas.calculate_count_matrix(count_matrix_file_name=path.join(indir, name+'_count_matrix.txt'))
    ps, svd = gwas.pop_struct_eigendecomp(cm)
    gwas.population_structure_formatter(ps, svd, number_of_pcs=2, 
                                        pop_struct_file_name=path.join(indir, name+'_structure_matrix.txt'))
    gwas.trait_formatter(trait_file_name=path.join(indir, name+'_phenotype_vector.txt'))
    gwas.calc_kinship_matrix(cm, minor_allele_fs, kinship_matrix_file_name=path.join(indir, name+'_kinship_matrix.txt'))
    gwas.hapmap_formatter(hapmap_file_name=path.join(indir, name+'_simulated_hapmap.txt'))
    gwas.single_gen_multi_rep_tassel_config(rep, 'gwas_pipeline.xml', output_prefix=name+'_out_')

# Run TASSEL at This Point

Contents of bash script to automate TASSEL via configuration files.

### simulated_mlm.sh

```bash
#!/bin/bash


echo "Run ID: $1, Number of Replicates $2"
run_id=$1
number_of_replicates=$2
final_rep_index="$((number_of_replicates - 1))"

echo "Beginning TASSEL analysis of Run ID: $run_id"
echo "Number of Replicates: $number_of_replicates"
echo "First configuration file: small_0_gwas_pipeline.xml"onca
echo "Final configuration file: small_"$final_rep_index"_gwas_pipeline.xml"

for i in `seq 0 $final_rep_index`
do
    config_file_name=$run_id$i"_gwas_pipeline.xml"
    echo "$config_file_name"
    ./run_pipeline.pl -Xmx6g -configFile $config_file_name
done


```

### Example output: 
+ small_0_out_1.txt
+ small_0_out_2.txt
+ small_0_out_3.txt
+ small_1_out_1.txt
+ ...
+ small_9_out_3.txt

# Use R Qvalue package to get Qvalues

Contents of R script to obtain qvalues for p column of TASSEL results

```R
#!/usr/bin/env Rscript

library(qvalue)
library(ggplot2)
library(gap)

args = commandArgs(trailingOnly=TRUE)

# test to determine if the file name parameter is supplied to the script
if (length(args)==0) {
  stop("At least one argument must be suppled (input file).\n", call.=FALSE)
}
#setwd("/home/vakanas/tassel-5-standalone/output")  

run_id = args[1]
file_name_match_pattern = paste(run_id, "(.*)_2.txt", sep='')
file_names = list.files(pattern = file_name_match_pattern)

for(n in file_names) {
    print(n)
    input_file_name = n
    run_id_prefix_terminus = nchar(input_file_name) - 5
    run_id_prefix = substring(input_file_name, 1, run_id_prefix_terminus)
    output_file_name = paste(run_id_prefix, 'q_values.txt', sep='')
    print(output_file_name)
    results_header = scan(input_file_name, what="character", nlines=1, sep="\t")
    gwas_results = read.table(input_file_name, header=F, row.names = NULL, skip=2)
    colnames(gwas_results) = results_header
    pvalues = gwas_results$p
    qobj = qvalue(p = pvalues)
    qvalues = data.frame(qobj$qvalues)
    colnames(qvalues) = "q"
    rownames(qvalues) = gwas_results$Marker
    write.table(qvalues, output_file_name, sep="\t")
}

```

# Analysis of TASSEL Results: Comutation of Power & FPR

## Subsetting Raw TASSEL Results and Data Storage
    Each replicate has an associated set of TASSEL output files. The raw
    results are modified and stored in the run's HDF5 file

## Statistical Power and False Positive Rate

In [64]:
qv = pd.read_csv('/home/vakanas/tassel-5-standalone/small_0_out_q_values.txt', sep='\t')

In [65]:
qv.ix[qv.q < 0.05]

,q
899,0.0004
1446,0.0037


In [66]:
qtl

[97,
 108,
 172,
 225,
 310,
 338,
 433,
 505,
 520,
 606,
 731,
 899,
 1145,
 1159,
 1190,
 1207,
 1349,
 1350,
 1413,
 1446]

In [69]:
sum(allele_effects[np.array(qv.ix[qv['q'] < 0.05].index), 2] > 0)

2

In [71]:
qv2 = pd.read_csv('/home/vakanas/tassel-5-standalone/small_1_out_q_values.txt', sep='\t')

In [75]:
allele_effects[np.array(qv2.ix[qv2.q < 0.05].index), 2]

array([ 0.411])

In [63]:
qtl

[97,
 108,
 172,
 225,
 310,
 338,
 433,
 505,
 520,
 606,
 731,
 899,
 1145,
 1159,
 1190,
 1207,
 1349,
 1350,
 1413,
 1446]

In [ ]:
allele_effects[np.array(qv2.ix[qv2['q'] < 0.05].index), 2]

In [ ]:
sum(qv2['q'] < 0.05)

In [79]:
list(small_data['tassel/test/replicate'])

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [80]:
del small_data['tassel/test/']

In [81]:
power_fprs = small.calculate_power_false_positive_rate(qtl, allele_effects, 10, hdf5_file=small_data)

In [82]:
power_fprs

array([[ 0.  ,  0.1 ,  0.  ],
       [ 1.  ,  0.05,  0.  ],
       [ 2.  ,  0.15,  0.  ],
       [ 3.  ,  0.1 ,  0.  ],
       [ 4.  ,  0.1 ,  0.  ],
       [ 5.  ,  0.15,  0.  ],
       [ 6.  ,  0.  ,  0.  ],
       [ 7.  ,  0.05,  0.  ],
       [ 8.  ,  0.1 ,  0.  ],
       [ 9.  ,  0.05,  0.  ]])

## Allele Effects: Estimated vs. Actual
    Purpose of this table to compare TASSEL's effect size estimates
    with the known true values we assigned.

In [83]:
small.genotypic_effects_table(sample_library, ae_array,
                          segregating_loci, qtl, hdf5_file=small_data)

TypeError: Object dtype dtype('O') has no native HDF5 equivalent

In [ ]:
raw_est_fx = pd.read_csv('/home/vakanas/tassel-5-standalone/small_0_out_3.txt', sep='\t')

In [ ]:
fx_by_marker = dict(list(raw_est_fx.groupby('Marker')))

In [ ]:
qtl

In [ ]:
fx_by_marker[101][Allele]

In [ ]:
sim.stat(sample_library[0][0], genoFreq=list(segregating_loci))

In [ ]:
for i in range(10):
    pop = sample_library[i][0]
    sim.stat(pop, genoFreq=list(segregating_loci))

In [90]:
table_columns = [
    'Marker',
    'G1',
    'P(G1)',
    'e[G1]*',
    'e[G1]',
    '|e[G1]* - e[G1]|',
    'G2',
    'P(G2)',
    'e[G2]*',
    'e[G2]',
    '|e[G2]* - e[G2]|',
    'G3',
    'P(G3)',
    'e[G3]*',
    'e[G3]',
    '|e[G3]* - e[G3]|'
]

In [ ]:
for rep_idx in range(10):
    raw_effect_estimates = pd.read_csv('/home/vakanas/tassel-5-standalone/small_'+str(rep_idx)+'_out_3.txt',
                                      sep='\t')
    grouped_data = dict(list(raw_effect_estimates.groupby('Marker')))

In [86]:
raw_effect_estimates = pd.read_csv('/home/vakanas/tassel-5-standalone/small_0_out_3.txt', 
                                   sep='\t')

In [ ]:
sample.indInfo('replicate')[0]

In [ ]:
#raw_effect_estimates.drop(labels=['Trait', 'Locus', 'Site', 'Obs'], axis=1, inplace=True)

In [87]:
grouped = raw_effect_estimates.groupby('Marker')

In [88]:
mapping_of_frames = dict(np.array(grouped))

In [91]:
genotypic_effects = pd.DataFrame(np.zeros((943, len(table_columns))), index=segregating_loci, columns=table_columns)

In [92]:
genotypic_effects.Marker = segregating_loci

In [93]:
snp_to_int = small.snp_to_int

In [94]:
iupac_genotype_codes = small.iupac_genotype_codes

In [95]:
sim.stat(sample, genoFreq=list(segregating_loci))
for locus in qtl:
    for idx, allele in enumerate(mapping_of_frames[locus].Allele):
        current_genotype = 'G'+str(idx+1)
        estimated_genotypic_effect_key = 'e[G'+str(idx+1)+']*'
        estimated_effect =  float(mapping_of_frames[locus].ix[mapping_of_frames[locus].ix[:, 'Allele'] == allele].Effect)
        true_genotypic_effect_key = 'e[G'+str(idx+1)+']'
        true_genotypic_effect = ae_array[locus, snp_to_int[iupac_genotype_codes[allele][0]]] + ae_array[locus, snp_to_int[iupac_genotype_codes[allele][1]]]
        genotypic_effects.ix[locus, current_genotype] = iupac_genotype_codes[allele]
        genotypic_effects.ix[locus, estimated_genotypic_effect_key] = estimated_effect
        genotypic_effects.ix[locus, true_genotypic_effect_key] = true_genotypic_effect
        frq_genotype_key = 'P(G'+str(idx+1)+')'
        frq_genotype = sample.dvars().genoFreq[locus][snp_to_int[iupac_genotype_codes[allele][0]], 
                                                      snp_to_int[iupac_genotype_codes[allele][1]]]
        genotypic_effects.ix[locus, frq_genotype_key] = frq_genotype
        '|e[G1]* - e[G1]|'
        abs_difference_key = '|e['+current_genotype+']* - e['+current_genotype+']|'
        abs_difference = abs(estimated_effect - true_genotypic_effect)
        genotypic_effects.ix[locus, abs_difference_key] = abs_difference

In [ ]:
mapping_of_frames[275]

In [ ]:
sample.dvars().genoFreq[275]

In [ ]:
genotypic_effects.ix[qtl]

In [ ]:
genotypic_effects.ix[13]

In [ ]:
genotypic_effects.cov()

In [ ]:
sim.stat(sample, genoFreq=list(segregating_loci))

In [ ]:
sample.dvars().genoFreq[1198]

In [ ]:
mapping_of_frames[13].ix[mapping_of_frames[13].ix[:, 'Allele'] == 'A']

In [ ]:
small_data['tassel/test/replicate']

In [ ]:
allele_effects

In [ ]:
small_data['allele/effects'].attrs['column_labels'] = list(map(np.string_, ['locus', 'alpha',
                                                                        'alpha_effect', 'omega',
                                                                        'omega_effect']))

In [ ]:
small_data['allele/effects'].attrs['column_labels']

In [ ]:
raw_effect_estimates

In [ ]:
table_columns

In [ ]:
mapped_strings = list(map(np.str_, table_columns))

In [ ]:
type(mapped_strings[0])

In [ ]:
small_data['tassel/test']

In [ ]:
first_genotype = raw_effect_estimates.ix[::3]

In [ ]:
936+935+935

In [ ]:
first_genotype.shape

In [ ]:
second_genotype = raw_effect_estimates.ix[1::3]

In [ ]:
second_genotype.shape

In [ ]:
third_genotype = raw_effect_estimates.ix[2::3]

In [ ]:
third_genotype.shape

In [ ]:
flattened_effect_table

In [ ]:
first_homozygo = np.array(raw_effect_estimates)[0::3, 1:]

In [ ]:
frame_first_homozygo = pd.DataFrame(first_homozygo, index=first_homozygo[:, 0])

In [ ]:
frame_first_homozygo.ix[13, 3]

In [ ]:
ae_array[13, snp_to_int[iupac_genotype_codes[frame_first_homozygo.ix[13, 3]][0]]]

In [ ]:
true_genotypic_effects = np.zeros((1478, 6, 6))

In [ ]:
true_genotypic_effects[13, 2, 2] = ae_array[13, snp_to_int[iupac_genotype_codes[frame_first_homozygo.ix[13, 3]][0]]]*2

In [ ]:
true_genotypic_effects[13]

In [ ]:
second_homozygo = np.array(raw_effect_estimates)[1::3, 1:]

In [ ]:
frame_second_homozygo = pd.DataFrame(second_homozygo, index=second_homozygo[:, 0])

In [ ]:
true_genotypic_effects = ae_array[13, snp_to_int[iupac_genotype_codes[frame_second_homozygo.ix[13, 3]][0]]]*2

In [ ]:
iupac_genotype_codes[frame_second_homozygo.ix[13, 3]]

In [ ]:
snp_to_int[iupac_genotype_codes[frame_second_homozygo.ix[13, 3]]]

In [ ]:
true_genotypic_effects[13, ]

In [ ]:
second_homozygo

In [ ]:
heterozygo = np.array(raw_effect_estimates)[2::3, 1:]

In [ ]:
heterozygo

In [ ]:
heterozygo

In [ ]:
iupac_genotype_codes = {
    'A': 'AA',
    0: 'A',
    'AA': 'A',
    1: 'C',
    'C': 'CC',
    'CC': 'C',
    2: 'G',
    'G': 'GG',
    'GG': 'G',
    3: 'T',
    'T': 'TT',
    'TT': 'T',
    'R': 'AG',
    'AG': 'R',
    'Y': 'CT',
    'CT': 'Y',
    'S': 'CG',
    'CG': 'S',
    'W': 'AT',
    'AT': 'W',
    'K': 'GT',
    'GT': 'K',
    'M': 'AC',
    'AC': 'M',
    5: '+',
    '+': '++',
    '++': '+',
    '0': '+-',
    '+-': '0',
    4: '-',
    '-': '--',
    '--': '-',
}

In [ ]:
snp_to_int = {
    'A': 0,
    'C': 1,
    'G': 2,
    'T': 3,
    '-': 4,
    '+': 5
}

In [ ]:
estimated_effect_sizes = np.zeros((943, 5))

In [ ]:
alphabetical_genotype = iupac_genotype_codes[first_homozygo[0, 4]]

In [ ]:
snp_to_int[alphabetical_genotype[0]]

In [ ]:
qtl

In [ ]:
raw_effect_estimates

In [ ]:
iupac_genotype_codes[iupac_genotype_codes[first_homozygo[13, 4]]]

In [ ]:
iupac_genotype_codes[first_homozygo[9, 4]]

In [ ]:
snp_to_int['G']

In [ ]:
ae_array[13, 2]*2

In [ ]:
iupac_genotype_codes['T']

In [ ]:
snp_to_int['T']

In [ ]:
allele_effects[13]

In [ ]:
ae_array[26, ]

In [ ]:
ae_array[qtl]

In [ ]:
even_effects

In [ ]:
odd_effects.shape

In [ ]:
allele_effects[qtl]

In [ ]:
rep_zero_afrqs = np.array(small_data['allele/frequency/replicate/0'])

In [ ]:
rep_zero_afrqs[qtl]

In [ ]:
qvalues.ix[qvalues['q'] < 0.05]

In [ ]:
def tassel_results_tables(gwas_file_name, q_values_file_name, 
                              minor_allele_frequency_table, 
                              quantitative_allele_table):
    raw_gwas_results = pd.read_csv(gwas_file_name, sep='\t')
    raw_gwas_results.drop(0, axis=0, inplace=True)
    raw_gwas_results.drop('Trait', axis=1, inplace=True)
    raw_gwas_results.index = np.array(list(map(int, raw_gwas_results.Marker)))
    q_values = pd.read_csv(q_values_file_name, sep='\t')
    q_values.index = np.array(list(map(int, raw_gwas_results.Marker)))
    raw_gwas_results = raw_gwas_results.join(q_values)
    
    assert minor_allele_frequency_table.index.dtype == raw_gwas_results.index.dtype, "Indexes of these tables are different"
    
    raw_gwas_results = raw_gwas_results.join(minor_allele_frequency_table.ix[raw_gwas_results.index, :])
    
    assert quantitative_allele_table.index.dtype == raw_gwas_results.index.dtype, "Indexes of these tables are different"
    
    raw_gwas_results = raw_gwas_results.join(quantitative_allele_table.ix[raw_gwas_results.index, :])
    return raw_gwas_results

In [ ]:
pwd

In [ ]:
cd /home/vakanas/tassel-5-standalone/output/

In [ ]:
ls

In [ ]:
mafrqs = pd.read_csv('epsilon_0_maf_table.txt', sep='\t', index_col=0)

In [ ]:
mafrqs

In [ ]:
qtad = pd.read_csv('epsilon_0_quant_allele_table.txt', sep='\t', index_col=0)

In [ ]:
qtad

In [ ]:
super_table = tassel_results_tables('epsilon_0_out_2.txt', 'epsilon_0_qvalues.txt', mafrqs, qtad)

In [ ]:
super_table.ix[super_table.q < 0.05]

In [ ]:
super_table.ix[super_table.alpha_effect > 0]

In [ ]:
mg.multiple_sample_analyzer(meta_populations, qtl, allele_effects, 
                            minor_alleles, concordant_segregating_loci)

In [ ]:
analyze.store_allele_effect_frequency_tables(meta_population, alleles, 
                                             qtl,
                                             exponential_allele_effects,
                                            run_id, 'exponential')

In [ ]:
loci_conversions = shelve.open(run_id+'_loci_conversions')
saegus_to_tassel_loci = {}
tassel_to_saegus_loci = {}
for idx, locus in enumerate(concordant_segregating_loci):
    saegus_to_tassel_loci[locus] = idx
    tassel_to_saegus_loci[idx] = locus
loci_conversions['saegus_to_tassel'] = saegus_to_tassel_loci
loci_conversions['tassel_to_saegus'] = tassel_to_saegus_loci
loci_conversions.close()

In [ ]:
seg_loc_storage = shelve.open('segregating_loci_storage')
seg_loc_storage['bacchus'] = concordant_segregating_loci
seg_loc_storage.close()

In [ ]:
int_to_snp = {0: 'A', 1: 'C', 2: 'G', 3: 'T', 4: '-', 5: '+'}
snp_to_int = {'A': 0, 'C': 1, '-': 4, 'G': 2, '+': 5, 'T': 3}
conv = shelve.open('synthesis_parameters')
conv['integer_to_snp'] = int_to_snp
conv['snp_to_integer'] = snp_to_int
conv.close()

In [ ]:
exponential_allele_effects_table = analyze.generate_allele_effects_table(qtl, alleles, 
                                                exponential_allele_effects, saegus_to_tassel_loci)

In [ ]:
analyze.remap_allele_frequency_table_loci(analyze.reload_allele_frequencies_table(run_id, 0, 250, 
                                                                                  'exponential'), 
                                          concordant_segregating_loci)

In [ ]:
analyze.write_multiple_sample_analyzer(sample_library, sample_sizes, qtl, alleles, 
                                       exponential_allele_effects, 0.7,  concordant_segregating_loci, 
                                       run_id=run_id, sub_run_id='_exponential', 
                                       allele_frequency_hdf=run_id+'_allele_frequency_storage.h5')

In [ ]:
multiple_sample_analyzer(meta_populations, qtl, allele_effects, minor_alleles, concordant_segregating_loci)

In [ ]:
import h5py

In [ ]:
with h5py.File('bia_allele_frequencies.hdf5') as biaf:
    reloaded_af = np.array(biaf[afname])

In [ ]:
fisegloc = list(concordant_segregating_loci)

In [ ]:
minor_allele_frequencies = reloaded_af[fisegloc]

In [ ]:
def write_super_tables(power_and_fpr_raw_data, sample_sizes, number_of_replicates, run_id, sub_run_id=''):
    for size in sample_sizes:
        for rep in range(number_of_replicates):
            name = run_id + '_' + sub_run_id + '_' + str(rep) + '_' + str(size) + '_super_table.txt'
            power_and_fpr_raw_data[size][rep].to_csv(name, sep='\t')

In [ ]:
expo_power_fpr_raw_data = analyze.collect_power_analysis_data(run_id, sample_sizes, number_of_replicates, concordant_segregating_loci, 'exponential')

In [ ]:
expo_power_fpr_raw_data[250]

In [ ]:
write_super_tables(expo_power_fpr_raw_data,
                  sample_sizes,
                  number_of_replicates,
                  'bacchus',
                  sub_run_id='exponential')

In [ ]:
expo_results, expo_true_positives, expo_false_positives = study.calculate_power_fpr(expo_power_fpr_raw_data, sample_sizes, 
                                                                             number_of_replicates, number_of_qtl)

In [ ]:
expo_results

In [ ]:
mean_and_stdev = pd.DataFrame([expo_results.mean(), expo_results.std()], index=['mean', 'stdev']).T
mean_and_stdev.to_csv('bacchus_exponential_mean_and_stdev_power_fpr.csv', sep='\t')

In [ ]:
geo_results

In [ ]:
geometric_allele_effects_table

In [ ]:
exponential_allele_effects_table

In [ ]:
expo_results.to_csv("bacchus_exponential_power_fpr_results.txt", sep='\t')

In [ ]:
mean_and_stdev = pd.DataFrame([geo_results.mean(), geo_results.std()], index=['mean', 'stdev']).T
mean_and_stdev.to_csv('full_icecrown_geometric_mean_and_stdev_power_fpr.txt', sep='\t')

In [ ]:
expo_results, expo_true_positives, expo_false_positives = full_icecrown.calculate_power_fpr(expo_power_fpr_raw_data,
                                                                                      sample_sizes,
                                                                                      number_of_replicates,
                                                                                      number_of_qtl)

In [ ]:
expo_results

In [ ]:
expo_results.to_csv('full_icecrown_exponential_power_fpr_results.txt', sep='\t')

In [ ]:
mean_and_stdev = pd.DataFrame([expo_results.mean(), expo_results.std()], index=['mean', 'stdev']).T
mean_and_stdev.to_csv('full_icecrown_exponential_mean_and_stdev_power_fpr.txt', sep='\t')

In [ ]:
write_super_tables(expo_power_fpr_raw_data, sample_sizes, number_of_replicates, run_id, 'exponential')

In [ ]:
geo_aggregate_estimated_actual = pd.DataFrame([np.array(geo_agg_estimated), np.array(geo_agg_actual)], index=['estimated', 'actual']).T

In [ ]:
geo_aggregate_estimated_actual['estimated'] = geo_aggregate_estimated_actual['estimated'].apply(np.fabs)

In [ ]:
geo_aggregate_estimated_actual

In [ ]:
geo_corr = geo_aggregate_estimated_actual['estimated'].corr(geo_aggregate_estimated_actual['actual'])

In [ ]:
geo_agg_estimated

In [ ]:
aggregate_estimated_actual

In [ ]:
geo_corr

In [ ]:
pwd

In [ ]:
geo_aggregate_estimated_actual.to_csv('full_icecrown_geometric_estimated_vs_actual_allele_effects.txt', sep='\t')

In [ ]:
agg_estimated = []
agg_actual = []

In [ ]:
for rep in reps:
    for size in sample_sizes:
        sutable = sutable_collection[rep][size]
        droppable = list(sutable.ix[sutable.ix[:, 'difference'] == 0.0].index)
        qtloci = sutable.drop(droppable, axis=0)
        agg_estimated.extend(list(qtloci['add_effect']))
        agg_actual.extend(list(qtloci['difference']))

In [ ]:
aggregate_estimated_actual = pd.DataFrame([np.array(agg_estimated), np.array(agg_actual)], index=['estimated', 'actual']).T

In [ ]:
aggregate_estimated_actual['estimated'] = np.fabs(aggregate_estimated_actual['estimated'])

In [ ]:
aggregate_estimated_actual

In [ ]:
correlation_actual_vs_effects = aggregate_estimated_actual['estimated'].corr(aggregate_estimated_actual['actual'])

In [ ]:
aggregate_estimated_actual.to_csv('full_icecrown_exponential_estimated_vs_actual_allele_effects.txt', sep='\t')

In [ ]:
aggregate_estimated_actual['estimated'] = np.fabs(aggregate_estimated_actual['estimated'])

In [ ]:
cd C:\tassel\output\full_icecrown\exponential

In [ ]:
expo_estimated_actual = pd.read_csv('full_icecrown_exponential_estimated_vs_actual_allele_effects.txt', sep='\t', index_col=0)

In [ ]:
expo_estimated_actual

In [ ]:
aggregate_estimated_actual

In [ ]:
, from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook

In [ ]:
output_notebook()

In [ ]:
aggregate_estimated_actual

In [ ]:
geo_x = aggregate_estimated_actual['estimated']
geo_y = aggregate_estimated_actual['actual']

In [ ]:
p = figure(title="Estimated vs Actual Allele Effects - Geometric Series", 
           title_text_font_size="16",
          x_range=(-0.2, 4))

In [ ]:
p.scatter(geo_x, y, x="Estimated", y="Actual")

p.xaxis.axis_label = "Estimated"
p.yaxis.axis_label = "Actual"

In [ ]:
show(p)

In [ ]:
expo

In [ ]:
p = figure(title="Estimated vs Actual Allele Effects - Geometric Series", title_text_font_size="16")

In [ ]:
expo_plot = figure(title="Estimated vs Actual Effects - Exponential(lambda=1)", 
                   title_text_font_size="16", 
                  x_range=(0, 4))

x = np.array(expo_estimated_actual['estimated'])
y = np.array(expo_estimated_actual['actual'])

expo_plot.xaxis.axis_label = "Estimated"
expo_plot.yaxis.axis_label = "Actual"

In [ ]:
expo_plot.scatter(x, y)

In [ ]:
show(expo_plot)

In [ ]:
from bokeh.io import hplot

In [ ]:
geo_plot = figure(title="Estimated vs Actual Allele Effects - Geometric Series", 
           title_text_font_size="16",
          x_range=(0, 4), y_range=(0, 4))

In [ ]:
geo_x = aggregate_estimated_actual['actual']
geo_y = aggregate_estimated_actual['estimated']

In [ ]:
geo_plot.xaxis.axis_label = "Actual"
geo_plot.yaxis.axis_label = "Estimated"
geo_plot.scatter(geo_x, geo_y, x="Actual", y="Estimated")

In [ ]:
expo_plot = figure(title="Estimated vs Actual Effects - Exponential(lambda=1)", 
                   title_text_font_size="16", 
                  x_range=(0, 4), y_range=(0, 4))

expo_x = np.array(expo_estimated_actual['actual'])
expo_y = np.array(expo_estimated_actual['estimated'])

expo_plot.xaxis.axis_label = "Actual"
expo_plot.yaxis.axis_label = "Estimated"
expo_plot.scatter(expo_x, expo_y)

In [ ]:
multi_plot = hplot(geo_plot, expo_plot)
show(multi_plot)

In [ ]:
output_file("multi_plot.png")

In [ ]:
ls